In [22]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
from pythainlp import word_tokenize
import deepcut
from gensim.models import Word2Vec
from pythainlp.tokenize import syllable_tokenize
from sklearn.metrics import confusion_matrix

In [2]:
def noise_removal(data):    
    removal = []
    for x in data:
        not_noise = x.index(':')+2
        removal.append(x[not_noise:])
    removal = [item.rstrip() for item in removal]
    return removal

def flat2gen(alist):
    for item in alist:
        if isinstance(item, list):
            for subitem in item: yield subitem
        else:
            yield item

In [3]:
f_input_test = open("input_exam2.txt",'r',encoding='utf-8')
# f_ans_test   = open("ans_test.txt",'r',encoding='utf-8')
data_input_test = f_input_test.readlines()
# data_ans_test   = f_ans_test.readlines()
data_input_test = noise_removal(data_input_test)
# data_ans_test   = noise_removal(data_ans_test)
# data_ans_test = [item.split(',') for item in data_ans_test]

In [6]:
bidirect_model = load_model("bidirect_model_pythai5.h5")

with open('word_tokenizer.pickle5', 'rb') as handle:
    word_tokenizer = pickle.load(handle)

In [8]:
max_sentence_length = 32
#model from deepcut is 32
#model from pythai is 30

In [9]:
word = [[w for w in deepcut.tokenize(s) if w!=' '] for s in data_input_test]

In [10]:
index = []
for idx,x in enumerate(word):
    temp = []
    for idx2,y in enumerate(x):
        if 'ตา' in y:
            if len(word_tokenize(y)) != 1:
                temp.append(idx2)
    index.append(temp)

In [11]:
for idx,_ in enumerate(index):
    for __ in _:
        word[idx][__] = word_tokenize(word[idx][__])
    word[idx] = list(flat2gen(word[idx]))

In [12]:
X_encoded = word_tokenizer.texts_to_sequences(word)
X_padded = pad_sequences(X_encoded,maxlen=max_sentence_length,padding="pre",truncating="post")
word_pred = bidirect_model.predict(X_padded)
word_pred = np.argmax(word_pred, axis=2)
word_pred = [[item for item in sublist if item>1] for sublist in word_pred]

In [13]:
#None = 1, E=2, P=3, T=4
for idx1,x in enumerate(word_pred):
    for idx2,y in enumerate(x):
        if(y==2):word_pred[idx1][idx2] = 'E'
        elif(y==3):word_pred[idx1][idx2] = 'P'
        elif(y==4):word_pred[idx1][idx2] = 'T'

In [14]:
for idx,_ in enumerate(word):
    print(idx)
    print(_)
    print(word_pred[idx])
    print('*'*50)

0
['"', 'ขอ', 'ตา', 'หน้า', 'อีก', 'ตาเดียว', '"', 'กู', 'เห็น', 'มึง', 'พูด', 'แบบ', 'นี้', 'มา', 'ทุก', 'ตา', '555', '+']
['T', 'T', 'P']
**************************************************
1
['การ', 'เดินหมาก', 'ตา', 'นี้', 'ทำ', 'ให้', 'ผม', 'มอง', 'โกะ', 'ใน', 'มุม', 'ใหม่']
['E']
**************************************************
2
['การ', 'ที่', 'ลูก', 'ได้', 'รด', 'น้ำ', 'ดำ', 'หัว', 'ปู่ย่าตายาย', 'จะ', 'ส่งเสริม', 'พัฒนา', 'การ', 'ด้าน', 'ใด', 'บ้าง']
['P']
**************************************************
3
['การ', 'ปิด', 'ตา', 'ตี', 'หม้อ', 'นิยม', 'เล่น', 'กัน', 'มา', 'นาน', 'แล้ว', 'โดย', 'เฉพาะ', 'ใน', 'เทศกาล', 'วัน', 'สงกรานต์', 'เพราะ', 'เป็น', 'การละเล่น', 'ที่', 'ชวน', 'ให้', 'สนุก', 'ทั้ง', 'คน', 'เล่น', 'และ', 'คน', 'ดู', 'ฝึก', 'ความ', 'จำ', 'การ', 'สังเกต', 'ทิศทาง', 'และ', 'การ', 'กะ', 'ระยะ', 'ทาง']
['E']
**************************************************
4
['การ', 'ผ่าตัด', 'เปลี่ยน', 'กระจก', 'ตา', 'สำเร็จ', 'เป็น', 'ครั้ง', 'แรก', 'นั้น', 'ไม่', 'ใช่', 'ตา'

In [15]:
problem = []
for idx,_ in enumerate(word_pred):
    if len(_) == 0 :
        problem.append(idx)

In [16]:
index = []
for x in problem:
    temp = []
    for idx,y in enumerate(word[x]):
        if 'ตา' in y:
            temp.append(idx)
    index.append(temp)

In [18]:
for x in problem:
    for _ in index:
        for __ in _:
            word[x][__] = syllable_tokenize(word[x][__])
        word[x] = list(flat2gen(word[x]))

In [19]:
X_encoded = word_tokenizer.texts_to_sequences(word)
X_padded = pad_sequences(X_encoded,maxlen=max_sentence_length,padding="pre",truncating="post")
word_pred = bidirect_model.predict(X_padded)
word_pred = np.argmax(word_pred, axis=2)
word_pred = [[item for item in sublist if item>1] for sublist in word_pred]
for idx1,x in enumerate(word_pred):
    for idx2,y in enumerate(x):
        if(y==2):word_pred[idx1][idx2] = 'E'
        elif(y==3):word_pred[idx1][idx2] = 'P'
        elif(y==4):word_pred[idx1][idx2] = 'T'

In [20]:
for idx,_ in enumerate(word):
    print(idx)
    print(_)
    print(word_pred[idx])
    print('*'*50)

0
['"', 'ขอ', 'ตา', 'หน้า', 'อีก', 'ตาเดียว', '"', 'กู', 'เห็น', 'มึง', 'พูด', 'แบบ', 'นี้', 'มา', 'ทุก', 'ตา', '555', '+']
['T', 'T', 'P']
**************************************************
1
['การ', 'เดินหมาก', 'ตา', 'นี้', 'ทำ', 'ให้', 'ผม', 'มอง', 'โกะ', 'ใน', 'มุม', 'ใหม่']
['E']
**************************************************
2
['การ', 'ที่', 'ลูก', 'ได้', 'รด', 'น้ำ', 'ดำ', 'หัว', 'ปู่ย่าตายาย', 'จะ', 'ส่งเสริม', 'พัฒนา', 'การ', 'ด้าน', 'ใด', 'บ้าง']
['P']
**************************************************
3
['การ', 'ปิด', 'ตา', 'ตี', 'หม้อ', 'นิยม', 'เล่น', 'กัน', 'มา', 'นาน', 'แล้ว', 'โดย', 'เฉพาะ', 'ใน', 'เทศกาล', 'วัน', 'สงกรานต์', 'เพราะ', 'เป็น', 'การละเล่น', 'ที่', 'ชวน', 'ให้', 'สนุก', 'ทั้ง', 'คน', 'เล่น', 'และ', 'คน', 'ดู', 'ฝึก', 'ความ', 'จำ', 'การ', 'สังเกต', 'ทิศทาง', 'และ', 'การ', 'กะ', 'ระยะ', 'ทาง']
['E']
**************************************************
4
['การ', 'ผ่าตัด', 'เปลี่ยน', 'กระจก', 'ตา', 'สำเร็จ', 'เป็น', 'ครั้ง', 'แรก', 'นั้น', 'ไม่', 'ใช่', 'ตา'

In [ ]:
# f = open("ans_pred.txt","w")
# for index,ans in enumerate(word_pred):
#     f.write(str(index+1)+'::')
#     for index2,class_ans in enumerate(ans):
#         f.write(class_ans)
#         if index2 != len(ans)-1:
#             f.write(',')
#     if len(word_pred)-1 != index:
#         f.write('\n')
# f.close()

In [24]:
cm = confusion_matrix(X_test, y_test)

NameError: name 'X_test' is not defined